In [1]:
import json
import seaborn as sns
import networkx as nx
import os
from networkx.algorithms import community
from Var_Redes import *
import freeman as fm
import json
import pandas as pd
import statsmodels.api as sm

In [2]:
def make_Eignvector(rede):
    ec = nx.eigenvector_centrality(rede, max_iter=100)
    return ec

def make_Degree(rede):
    dc = nx.degree_centrality(rede)
    return dc

def make_Betweenness(rede):
    bc = nx.betweenness_centrality(rede)
    return bc

def make_Closeness(rede):
    cc = nx.closeness_centrality(rede)
    return cc

def make_AssortividadeGrau(rede):
    c = nx.degree_assortativity_coefficient(rede)
    return c

def proportion(rede, nós, vizinhos):
    return 1 / rede.degree(nós)

def local_effsize(rede, nós, vizinhos):
    s = 1
    for k in rede.neighbors(nós):
        if rede.has_edge(k, vizinhos):
            s -= proportion(rede, nós, k)
    return s

def effsize(rede, nós):
    s = 0
    for vizinhos in rede.neighbors(nós):
        s += local_effsize(rede, nós, vizinhos)
    return s

def local_constraint(rede, nós, vizinhos):
    s = proportion(rede, nós, vizinhos)
    for k in rede.neighbors(nós):
        if rede.has_edge(k, vizinhos):
            s += proportion(rede, nós, k) * proportion(rede, k, vizinhos)
    return s**2

def constraint(rede, nós):
    if rede.degree(nós) == 0:
        return 2
    s = 0
    for vizinhos in rede.neighbors(nós):
        s += local_constraint(rede, nós, vizinhos)
    return s

def hierarchy(rede, nós):
    c = constraint(rede, nós)
    nós = rede.number_of_nodes()
    s = 0
    for vizinhos in rede.neighbors(nós):
        f = local_constraint(rede, nós, vizinhos) / (c / nós)
        s += f * log(f)
    return s / (nós * log(nós))

In [3]:
class Gerador:

    def __init__(self):
        self.json_data = {}
        self.creators_game_info = {}
        self.creators_infos = {}
        self.creators_game_count = {}

    def get_data(self):
        with open('creators_filtered_copy.json') as json_file:
            self.json_data = json.load(json_file)

    def map_relations(self):

        # CREATING A JSON_DATA COPY IN ORDER TO AVOID FUTURE REDUNDANT CONNECTIONS
        json_data_copy = self.json_data.copy()

        # ITERATE TRHOUGH DEVS TO GET GAMES DONE
        for creator_id in self.json_data:

            # POPULATING DEV'S GAMES
            creator_games_ids = []
            for game in self.json_data[creator_id]['games']:
                creator_games_ids.append(game["id"])
            self.creators_game_info[creator_id] = creator_games_ids

            # DELETING DEV FROM COPY TO AVOID REDUNDANCY
            del json_data_copy[creator_id]

            # GETTING CO-WORKERS
            co_workers = []
            for creator in json_data_copy:
                # AVOIDING UNNECESSARY ITERATIONS TO SAVE RESOURCES
                if creator in co_workers:
                    break

                # MUST VERIFY ALL GAMES
                for game in json_data_copy[creator]["games"]:
                    if game["id"] in self.creators_game_info[creator_id]:
                        # JUST LIKE LINE 35
                        if creator in co_workers:
                            break
                        else:
                            co_workers.append(creator)

                                           # [rating, lista com os co_workers]
            self.creators_infos[creator_id] = [self.json_data[creator_id]["rating"] ,co_workers]
            self.creators_game_count[creator_id] = [self.json_data[creator_id]["games_count"] ,co_workers]

#           print(self.creators_relationships)

    def generate_network(self):
#         if os.path.isfile("network_data.gml"):
#             os.remove("network_data.gml")

        # CREATING GRAPH LEVEL
        f = open("network_data.gml", "w")
        f.write("graph [\n")
        f.write("   directed 0\n")
        f.write("\n")

        # CREATING NODES
        for node in self.creators_infos:
            f.write("   node [\n")
            f.write("        id " + node + "\n")
            f.write(f'       label node_{node} \n')
            f.write("   ]\n")

        # CREATE CONNECTION
        for source in self.creators_infos:
            for target in self.creators_infos[source][1]:
                f.write("   edge [\n")
                f.write("       source " + source + "\n")
                f.write("       target " + target + "\n")
                f.write("   ]\n")

        f.write("]")
        f.close()


In [4]:
gerador = Gerador()
gerador.get_data()
gerador.map_relations()
gerador.generate_network()

In [16]:
g = fm.load('network_data.gml')
# Deixar os nós menores e fazer os nomes deles
# aparecer apenas quando o mouse passa por cima.
# g.set_all_nodes(size=10, labpos='hover')

# Deixar as arestas semi-transparentes para
# a visualização ficar um pouco mais leve.
# g.set_all_edges(color=(0, 0, 0, 0.25))

# Usar um algoritmo especial para posicionar
# os nós de acordo com grupos.
# g.move('kamada_kawai')

# g.draw()

In [6]:
# com = list(community.girvan_newman(g))

In [7]:
creators_eigens = make_Eignvector(g)

In [8]:
creators_ratings = {}

for source in gerador.creators_infos:
    creators_ratings[source] = gerador.creators_infos[source][0]
    
print(creators_ratings)

{'1': 4.36, '10': 4.386666666666667, '100': 4.15, '1000': 4.3500000000000005, '10001': 2.3000000000000003, '10002': 3.0525, '10003': 3.0525, '10004': 3.56, '10005': 3.56, '10008': 0.0, '10009': 0.0, '1001': 4.234999999999999, '10011': 0.0, '10012': 0.0, '10013': 4.22, '10014': 2.7399999999999998, '10015': 4.07, '10016': 4.07, '10017': 3.6025, '10018': 3.79, '10019': 3.0233333333333334, '1002': 3.345, '10020': 3.6966666666666668, '10022': 1.205, '10023': 0.0, '10024': 0.0, '10025': 1.6, '10026': 1.6, '10027': 1.0666666666666667, '10028': 1.6, '10029': 1.6, '1003': 3.448333333333333, '10030': 1.6, '10031': 0.8, '10032': 1.6, '10033': 1.6, '10034': 1.6, '10036': 0.0, '10037': 0.0, '10038': 1.05, '10039': 1.7875, '1004': 4.07, '10041': 0.0, '10042': 0.0, '10044': 0.0, '10046': 0.0, '10047': 0.0, '10048': 0.0, '10049': 0.0, '1005': 4.07, '10050': 0.0, '10051': 0.858, '10052': 0.0, '10053': 4.04, '10054': 3.67, '10055': 0.0, '10058': 0.6916666666666668, '10059': 1.3833333333333335, '1006': 2

In [9]:
creators_game_count = {}

for source in gerador.creators_game_count:
    creators_game_count[source] = gerador.creators_game_count[source][0]
    
print(creators_game_count)

{'1': 5, '10': 3, '100': 2, '1000': 6, '10001': 3, '10002': 4, '10003': 4, '10004': 1, '10005': 1, '10008': 1, '10009': 1, '1001': 2, '10011': 1, '10012': 1, '10013': 1, '10014': 3, '10015': 1, '10016': 1, '10017': 4, '10018': 1, '10019': 6, '1002': 2, '10020': 3, '10022': 6, '10023': 6, '10024': 4, '10025': 2, '10026': 2, '10027': 3, '10028': 2, '10029': 2, '1003': 6, '10030': 2, '10031': 4, '10032': 2, '10033': 2, '10034': 2, '10036': 1, '10037': 1, '10038': 4, '10039': 4, '1004': 1, '10041': 1, '10042': 2, '10044': 2, '10046': 1, '10047': 1, '10048': 1, '10049': 1, '1005': 1, '10050': 1, '10051': 5, '10052': 1, '10053': 1, '10054': 1, '10055': 1, '10058': 6, '10059': 3, '1006': 2, '10060': 5, '10062': 3, '10063': 4, '10064': 4, '10065': 1, '10066': 2, '10068': 2, '10069': 3, '1007': 1, '10071': 3, '10072': 3, '10073': 4, '10074': 3, '10075': 3, '10076': 3, '10077': 1, '10078': 1, '1008': 2, '10080': 2, '10081': 2, '10082': 2, '10083': 2, '10084': 1, '10085': 2, '10087': 1, '10088': 

In [10]:
print(creators_eigens)

{1: 5.318960597861982e-07, 10: 1.674833065060572e-07, 100: 7.14563850831774e-06, 1000: 1.2901500493655726e-06, 10001: 5.360877008967702e-07, 10002: 3.088547732914844e-08, 10003: 3.088547732914844e-08, 10004: 8.380428867031874e-34, 10005: 8.380428867031874e-34, 10008: 4.402269666876435e-61, 10009: 4.402269666876435e-61, 1001: 6.851632195274471e-07, 10011: 4.402269666876435e-61, 10012: 4.402269666876435e-61, 10013: 1.6395263511744204e-09, 10014: 5.9171651619928756e-08, 10015: 5.0085574661126857e-08, 10016: 5.0085574661126857e-08, 10017: 7.908627533150806e-05, 10018: 2.448771217257821e-05, 10019: 7.919979371876545e-05, 1002: 9.7200388572864e-07, 10020: 6.353227332670785e-05, 10022: 2.679994550684924e-09, 10023: 2.897180573744487e-10, 10024: 1.0240451801764975e-08, 10025: 4.454227223968711e-09, 10026: 4.454227223968711e-09, 10027: 7.491825276268305e-09, 10028: 4.454227223968711e-09, 10029: 4.454227223968711e-09, 1003: 9.734845744412318e-07, 10030: 4.454227223968711e-09, 10031: 4.4542272239

In [11]:
eigens = []
for key, value in creators_eigens.items():
    eigens.append(value)

In [12]:
ratings = []
for key, value in creators_ratings.items():
    ratings.append(value)

In [13]:
game_count = []
for key, value in creators_game_count.items():
    game_count.append(value)

In [14]:
# Primeiro parâmetro: Y
# Segundo parâmetro: X1, ..., Xn (não esqueça os colchetes duplos)
model = sm.OLS(ratings, eigens)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.002
Model:                            OLS   Adj. R-squared (uncentered):              0.002
Method:                 Least Squares   F-statistic:                              45.50
Date:                Fri, 13 Nov 2020   Prob (F-statistic):                    1.57e-11
Time:                        18:30:57   Log-Likelihood:                         -46030.
No. Observations:               19556   AIC:                                  9.206e+04
Df Residuals:                   19555   BIC:                                  9.207e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            17.1794      2.547      6.745      0.000      12.187      22.171
==============================================================================
Omnibus:                    84998.238   Durbin-Watson:                   0.452
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2358.532
Skew:                           0.075   Prob(JB):                         0.00
Kurtosis:                       1.305   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
# Primeiro parâmetro: Y
# Segundo parâmetro: X1, ..., Xn (não esqueça os colchetes duplos)
model = sm.OLS(ratings, game_count)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.436
Model:                            OLS   Adj. R-squared (uncentered):              0.436
Method:                 Least Squares   F-statistic:                          1.512e+04
Date:                Fri, 13 Nov 2020   Prob (F-statistic):                        0.00
Time:                        18:30:57   Log-Likelihood:                         -40451.
No. Observations:               19556   AIC:                                  8.090e+04
Df Residuals:                   19555   BIC:                                  8.091e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.7017      0.006    122.975      0.000       0.690       0.713
==============================================================================
Omnibus:                     9074.513   Durbin-Watson:                   1.080
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1239.239
Skew:                           0.264   Prob(JB):                    7.99e-270
Kurtosis:                       1.885   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""